In [1]:
import cv2
import numpy as np
from PIL import Image
import copy

In [4]:
cap = cv2.VideoCapture(1)
count = 0
font = cv2.FONT_HERSHEY_SIMPLEX
length = 5
frames = [None] * length

In [ ]:
while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_np = np.array(frame)
    backup = np.zeros(frame.shape)
    if count < length:
        frames[count] = frame_np
    else:
        backup = frames[count % length]
        frames[count % length] = frame_np
    frame_np = frame_np.astype('int32')
    backup = backup.astype('int32')
    diff = np.abs(frame_np - backup)
    diff = diff.astype('uint8')
    frame_np = frame_np.astype('uint8')
    backup = backup.astype('uint8')
    diff[0][0] = 0
    diff[0][1] = 255
    diff_c = copy.deepcopy(diff)
    diff_c[diff_c > 220] = 0.0
    cv2.imshow('diff abs', diff)
    cv2.imshow('diff abs + filter', diff_c)
    cv2.imshow('frame', frame_np)
    count += 1
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [5]:
diff.shape

(480, 640)

In [ ]:
cv2.imshow('1', diff[150:200, 150:200])
cv2.imshow('2', diff_c)
cv2.imshow('3', frame_np)
cv2.waitKey(0)
cv2.destroyAllWindows()
print (np.average(diff))

In [ ]:
backup_cutout = backup[150:200, 150:200]
frame_np_cutout = frame_np[150:200, 150:200]
diff_cutout = diff[150:200, 150:200]
#print (diff_cutout.dtype)
print (diff_cutout)
print ('\n')
print (backup_cutout)
print ('\n')
print(frame_np_cutout)
print ('\n')
print (frame_np_cutout - backup_cutout)

In [ ]:
while True:
    ret, frame = cap.read()
    a = np.array(frame)
    b = np.average(a)
    print (a.shape)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()